In [ ]:
import requests
import geopandas as gpd
import numpy as np
import shapely
import pandas as pd
import json

In [ ]:
endpoint = 'https://api.plastic.watch.earthrise.media/sites'
response = requests.get(endpoint, params={'limit':1500})
data = response.json()

In [ ]:
country_endpoint = 'https://api.dev.plastic.watch.earthrise.media/countries'
response = requests.get(country_endpoint, params={'limit':1500})
boundaries = response.json()

In [ ]:
gdf = gpd.GeoDataFrame.from_features(data['features'])
gdf['country code'] = ['' for _ in range(len(gdf))]
gdf.head()

In [ ]:
for feature in data['features']:
    site = shapely.geometry.Point(feature['geometry']['coordinates'])
    for country in boundaries['features']:
        poly = shapely.geometry.shape(country['geometry'])
        if poly.contains(site):
            gdf.loc[gdf['geometry'] == site, 'country code'] = country['properties']['Code']

In [ ]:
gdf['country']

In [ ]:
code_to_name= {}
for country in boundaries['features']:
    properties = country['properties']
    code_to_name[properties['Code']] = properties['LongName']

In [ ]:
country_stats = {}
for country in np.unique(gdf['country code']):
    if len(country) > 0:
        sites = gdf[gdf['country code'] == country]
        country_dict = {
            "totalSites": len(sites),
            "name": code_to_name[country],
            "surface": {
                "total (ha)": pd.to_numeric(sites['median area'], errors='coerce').sum() * 100,
                "above 1 ha": sum(pd.to_numeric(sites['median area'], errors='coerce') * 100 > 1),
                "above 2 ha": sum(pd.to_numeric(sites['median area'], errors='coerce') * 100 > 2),
            },
            "water": {
                "under 0.5 km": sum(sites['distance to waterway(m)'].astype(float) < 500),
                "under 1.0 km": sum(sites['distance to waterway(m)'].astype(float) < 1000),
                "under 2.0 km": sum(sites['distance to waterway(m)'].astype(float) < 2000),
            },
            "population": {
                "within 1 km": sum(sites['population - 1km'].astype(float)),
                "within 5 km": sum(sites['population - 5km'].astype(float)),
            }
        }
        country_stats[country] = country_dict
        country_stat_endpoint = f'https://api.dev.plastic.watch.earthrise.media/countries/{country}/stats'
        # API endpoint is currently read only
        #r = requests.put(country_stat_endpoint, country_dict)


In [ ]:
with open('../../data/site_metadata/country_stats.geojson', 'w') as f:
    json.dump(country_stats, f, indent=4)